In [ ]:
# 

In [ ]:
import os
import glob

import pandas as pd

import nibabel as nib
from nilearn import image
from nilearn import masking
from nilearn import plotting

In [ ]:
# Load an image from a single brain:
def img_load(folder, img):
    img_path = os.path.join(folder, img)
    fmri_image = nib.load(img_path)
    return fmri_image

In [ ]:
###################################################################################################################

### Holiga et al., 2019 - Patients with autism spectrum disorders display reproducible functional connectivity alterations [ https://www.science.org/doi/10.1126/scitranslmed.aat9223 ]

In [ ]:
# Load the resampled ASC>TD Holiga mask

In [ ]:
asc_td_mask_folder = "/Users/mishodimitrov/Downloads/PhD/Analysis/CBD/Data/Masks/Holiga_et_al/ASC_TD"

In [ ]:
asc_td_mask = img_load(asc_td_mask_folder, 'asc_td.nii.gz')

In [ ]:
plotting.plot_img(asc_td_mask)

In [ ]:
# Load the resampled TD>ASC Holiga mask

In [ ]:
td_asc_mask_folder = "/Users/mishodimitrov/Downloads/PhD/Analysis/CBD/Data/Masks/Holiga_et_al/TD_ASC"

In [ ]:
td_asc_mask = img_load(td_asc_mask_folder, 'td_asc.nii.gz')

In [ ]:
plotting.plot_img(td_asc_mask)

In [ ]:
###################################################################################################################

In [ ]:
## Generate an intersection mask

In [ ]:
img_folder = '/Users/mishodimitrov/Downloads/PhD/Analysis/CBD/Static_FC/DC_data/wDC_Z'

In [ ]:
# Get the correct ID's

In [ ]:
os.chdir('/Users/mishodimitrov/Downloads/PhD/Analysis/CBD/Data/Covariates')
covariate_sheet = pd.read_excel('Covariates.xlsx', sheet_name=0, engine='openpyxl')
covariate_sheet

In [ ]:
id_list = [covariate_sheet.iloc[:,0][i] for i in range(covariate_sheet.iloc[:,0].shape[0])]
id_list 

In [ ]:
img_list = ['MAGACAN' + id_list[i] + '_wDC_Z.nii' for i in range(len(id_list))]
img_list

In [ ]:
def binarise_append_intersect(img_folder, img_list, intersection_thr):
    img_list_bin = []
    # binarise images and append to list
    for i in range(len(img_list)):
        wDC_img = img_load(img_folder, img_list[i])
        #print('Image loaded.')
        wDC_img_bin = image.binarize_img(wDC_img, threshold=0, mask_img=None)
        #print('Image binarised.')
        img_list_bin.append(wDC_img_bin)
        #print('Image added to the list.')
    # create an intersection mask of all images in the list
    intersection_mask = masking.intersect_masks(img_list_bin, threshold=intersection_thr, connected=False)
    print('Intersection mask created.')
    return intersection_mask

In [ ]:
intersection_mask = binarise_append_intersect(img_folder, img_list, 0.85)

In [ ]:
plotting.plot_img(intersection_mask)

In [ ]:
# Save the intersection mask
os.chdir('/Users/mishodimitrov/Downloads/PhD/Analysis/CBD/Static_FC/Analysis/Step_2_Mean_wDC')
nib.save(intersection_mask, "intersection_mask_0.85.nii.gz")

In [ ]:
###################################################################################################################

In [ ]:
# Generate 2 final Holiga et al. masks, made from intersecting the intersection mask and the 2 masks, respectively
# N.B. After looking at the masks, I've decided to stick with the 0.85 one
final_asc_td_mask_list = [intersection_mask, asc_td_mask]
final_asc_td_mask = masking.intersect_masks(final_asc_td_mask_list, threshold=1, connected=False)
final_td_asc_mask_list = [intersection_mask, td_asc_mask]
final_td_asc_mask = masking.intersect_masks(final_td_asc_mask_list, threshold=1, connected=False)

In [ ]:
plotting.plot_img(final_asc_td_mask)

In [ ]:
plotting.plot_img(final_td_asc_mask)

In [ ]:
# Save the final Holiga et al. masks
os.chdir('/Users/mishodimitrov/Downloads/PhD/Analysis/CBD/Static_FC/Analysis/Step_2_Mean_wDC')
nib.save(final_asc_td_mask, "final_asc_td_mask.nii.gz")
nib.save(final_td_asc_mask, "final_td_asc_mask.nii.gz")